In [1]:
import csv
import sqlite3
import os
from pathlib import Path

def csv_to_sqlite(csv_file, sqlite_file=None, table_name=None):
    """
    Convert a CSV file to a SQLite database.
    
    Args:
        csv_file: Path to the input CSV file
        sqlite_file: Path to the output SQLite file (optional, defaults to CSV name with .db extension)
        table_name: Name of the table in SQLite (optional, defaults to CSV filename without extension)
    """
    # Set default sqlite_file if not provided
    if sqlite_file is None:
        sqlite_file = Path(csv_file).stem + '.db'
    
    # Set default table_name if not provided
    if table_name is None:
        table_name = Path(csv_file).stem.replace('-', '_').replace(' ', '_')
    
    # Check if CSV file exists
    if not os.path.exists(csv_file):
        raise FileNotFoundError(f"CSV file not found: {csv_file}")
    
    # Read CSV file
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        headers = next(reader)
        rows = list(reader)
    
    # Create SQLite connection
    conn = sqlite3.connect(sqlite_file)
    cursor = conn.cursor()
    
    # Clean column names (remove special characters, replace spaces with underscores)
    clean_headers = [h.strip().replace(' ', '_').replace('-', '_').replace('.', '_') 
                     for h in headers]
    
    # Create table with all columns as TEXT (SQLite will handle type affinity)
    columns = ', '.join([f'"{col}" TEXT' for col in clean_headers])
    cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
    cursor.execute(f'CREATE TABLE "{table_name}" ({columns})')
    
    # Insert data
    placeholders = ', '.join(['?' for _ in clean_headers])
    insert_query = f'INSERT INTO "{table_name}" VALUES ({placeholders})'
    cursor.executemany(insert_query, rows)
    
    # Commit and close
    conn.commit()
    conn.close()
    
    print(f"✓ Successfully converted '{csv_file}' to '{sqlite_file}'")
    print(f"  Table name: '{table_name}'")
    print(f"  Rows imported: {len(rows)}")
    print(f"  Columns: {len(clean_headers)}")

# Example usage
if __name__ == "__main__":
    
    # Example 3: Only specify table name
    csv_to_sqlite('data.csv', table_name='real_estate')

✓ Successfully converted 'data.csv' to 'data.db'
  Table name: 'real_estate'
  Rows imported: 511
  Columns: 14


In [4]:
import sqlite3

def extract_schema(sqlite_file):
    """
    Extract and display the schema from a SQLite database.
    
    Args:
        sqlite_file: Path to the SQLite database file
    """
    conn = sqlite3.connect(sqlite_file)
    cursor = conn.cursor()
    
    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    print(f"Database: {sqlite_file}")
    print("=" * 60)
    
    for table in tables:
        table_name = table[0]
        print(f"\nTable: {table_name}")
        print("-" * 60)
        
        # Get table schema
        cursor.execute(f"PRAGMA table_info('{table_name}');")
        columns = cursor.fetchall()
        
        print(f"{'Column':<20} {'Type':<15} {'Not Null':<10} {'Default':<15}")
        print("-" * 60)
        
        for col in columns:
            col_id, col_name, col_type, not_null, default_val, pk = col
            not_null_str = "YES" if not_null else "NO"
            default_str = str(default_val) if default_val is not None else ""
            pk_marker = " (PK)" if pk else ""
            
            print(f"{col_name:<20} {col_type:<15} {not_null_str:<10} {default_str:<15}{pk_marker}")
        
        # Get row count
        cursor.execute(f"SELECT COUNT(*) FROM '{table_name}';")
        row_count = cursor.fetchone()[0]
        print(f"\nTotal rows: {row_count}")
    
    conn.close()

def get_create_statements(sqlite_file):
    """
    Get the CREATE TABLE statements from a SQLite database.
    
    Args:
        sqlite_file: Path to the SQLite database file
    """
    conn = sqlite3.connect(sqlite_file)
    cursor = conn.cursor()
    
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")
    statements = cursor.fetchall()
    
    print(f"CREATE TABLE Statements for: {sqlite_file}")
    print("=" * 60)
    
    for stmt in statements:
        if stmt[0]:
            print(f"\n{stmt[0]};")
    
    conn.close()

# Example usage
if __name__ == "__main__":
    # Extract schema in readable format
    extract_schema('olist.sqlite')
    
    print("\n" + "=" * 60)
    print("\n")
    
    # Get CREATE TABLE statements
    get_create_statements('olist.sqlite')

Database: olist.sqlite

Table: product_category_name_translation
------------------------------------------------------------
Column               Type            Not Null   Default        
------------------------------------------------------------
product_category_name TEXT            NO                        
product_category_name_english TEXT            NO                        

Total rows: 71

Table: sellers
------------------------------------------------------------
Column               Type            Not Null   Default        
------------------------------------------------------------
seller_id            TEXT            NO                        
seller_zip_code_prefix INTEGER         NO                        
seller_city          TEXT            NO                        
seller_state         TEXT            NO                        

Total rows: 3095

Table: customers
------------------------------------------------------------
Column               Type            N